In [ ]:
pip install openpyxl

In [ ]:
import torch
import random

In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("SI2M-Lab/DarijaBERT")
model = AutoModel.from_pretrained("SI2M-Lab/DarijaBERT")
model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/307 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/836M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at SI2M-Lab/DarijaBERT and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(80000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [ ]:
import pandas as pd

ds = pd.read_excel(r'/content/sample_data/neg_comments.xlsx')
comments = ds['Comment'].tolist()

# ***2. Random Deletion***

In [ ]:
def random_deletion(comment, deletion_prob=0.2):
    """
    Supprimer aléatoirement un ou plusieurmot.
    """
    words = comment.split()

    if len(words) > 3:
        new_words = words.copy()
        #s'assurer que au moins un token est supprimé
        while len(new_words) == len(words):
            new_words = [word for word in words if random.random() > deletion_prob]
    else:
        new_words = " "

    new_sentence = ' '.join(new_words)
    return new_sentence

In [ ]:
def get_comment_embedding(comment):

    inputs = tokenizer(comment, return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)

    embeddings = outputs.last_hidden_state  # [batch_size, seq_len, hidden_dim]

    # Ignorer CLS et SEP et prendre la moyenne des embeddings restants
    mask = inputs["attention_mask"].unsqueeze(-1)  # [batch_size, seq_len, 1]
    sum_embeddings = torch.sum(embeddings * mask, dim=1)
    mean_embedding = sum_embeddings / mask.sum(dim=1)  # Moyenne pondérée

    #Retourne l'embedding moyen d'une phrase après passage par le modèle darija bert
    return mean_embedding.squeeze(0)  # Retirer la dimension batch

In [ ]:
from torch.nn.functional import cosine_similarity

def top_k_variants(comment, k=10, deletion_prob=0.2):
    #Génèrer 10 variants du commentaire et sélectionner le meilleur selon la cosine similarity.
    original_embedding = get_comment_embedding(comment)

    variants = [random_deletion(comment, deletion_prob) for _ in range(k)]
    best_comment = comment
    best_score = -1

    for variant in variants:
        variant_embedding = get_comment_embedding(variant)
        score = cosine_similarity(original_embedding, variant_embedding,dim=0).item()

        if score > best_score:
            best_comment = variant
            best_score = score

    return best_comment

In [ ]:
augmentedComments_by_RD =[]
for comment in comments:
    augmentedcomment = top_k_variants(comment)
    augmentedComments_by_RD.append(augmentedcomment)



In [ ]:
augmentedComments_by_RD = [item for item in augmentedComments_by_RD if item != " "]

In [ ]:
data = {"Augmented Comment": augmentedComments_by_RD}
df = pd.DataFrame(data)

df

In [ ]:
df.to_excel('augmentednegcomments_byRD.xlsx')